In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch.nn.functional as F
import pandas as pd
import torch
import math

In [3]:
target_sentences_df = pd.read_csv('/content/drive/MyDrive/PhD/target_sentences.csv')
target_sentences_df

,Text
0,još malo pa će izbori
1,naoblačilo se kao da će padati kiša
2,naša kola su stara 10 godina
3,otišao je u inostranstvo prošle godine i još s...
4,telefoni zvone po ceo dan i tebe traže
...,...
62,pojavio se u poslednjem trenutku
63,sanjao sam te pre neku noć
64,premijer je obećao viši standard
65,sutra je doček Nove Godine


In [4]:
vocabulary_df = pd.read_csv('/content/drive/MyDrive/PhD/wordlist_classlawiki_sr_cleaned.csv')
vocabulary_df

,word,freq
0,biti,5835707
1,u,4289454
2,i,2848044
3,na,1602609
4,sebe,1544095
...,...,...
336153,"1,77.",5
336154,"0,038",5
336155,"0,036",5
336156,"0,032",5


In [5]:
pip install transformers

In [6]:
pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 20.0 MB/s eta 0:00:00


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

model_name = 'gpt2'
# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
import stanza

# Initialize the Stanza pipeline for Serbian
stanza.download('sr')  # Ensure the model is downloaded
nlp = stanza.Pipeline('sr')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sr (Serbian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sr (Serbian):
| Processor | Package      |
----------------------------
| tokenize  | set          |
| pos       | set_nocharlm |
| lemma     | set_nocharlm |
| depparse  | set_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be load

**Similarity Functions**

In [9]:
def non_context_embedding(word, vord, model, tokenizer):

  # Tokenize and get embedding for the target word
  word_input_ids = tokenizer.encode(word, return_tensors='pt')

  with torch.no_grad():
    output = model(word_input_ids)
    # Access hidden_states and get the last layer
    word_embedding = output[0].mean(dim=1)

  vocab_input_ids = tokenizer.encode(vord, return_tensors='pt')
  with torch.no_grad():
    output = model(vocab_input_ids)
    # Access hidden_states and get the last layer
    vocab_embedding = output[0].mean(dim=1)

  # Compute cosine similarity and normalize
  similarity = 0.5 * (F.cosine_similarity(word_embedding, vocab_embedding).item() + 1)

  return similarity

In [10]:
def get_pos_for_word_at_index(word, sentence, index):
    '''
    Return POS tag for a single word at index in sentence.
    '''
    # Process the sentence
    doc = nlp(sentence)

    # Loop through the tokens to find the POS tag at the specified index
    for sentence in doc.sentences:
        for token in sentence.tokens:
            if token.text == word and token.words[0].id == index+1:
                return token.words[0].upos  # Return the universal POS tag for the word at the given index

    return None  # Return None if the word at the specified index is not found

In [11]:
def pos_tags_similarity(word1, word2, sentence, ind):
  '''
  Calculate POS tag similarity between word1 and word2 in sentnece at ind.
  '''

  words = sentence.split(" ")

  words[ind] = word1
  p1 = get_pos_for_word_at_index(word1, " ".join(words), ind)

  words[ind] = word2
  p2 = get_pos_for_word_at_index(word2, " ".join(words), ind)

  if p1 == p2:
    return 1
  else:
    return 0

In [12]:
def levenshtein_distance(str1, str2):
    # Create a distance matrix
    dp = [[0] * (len(str2) + 1) for _ in range(len(str1) + 1)]

    # Initialize the matrix
    for i in range(len(str1) + 1):
        dp[i][0] = i  # Deletion
    for j in range(len(str2) + 1):
        dp[0][j] = j  # Insertion

    # Calculate distances
    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            cost = 0 if str1[i - 1] == str2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,      # Deletion
                           dp[i][j - 1] + 1,      # Insertion
                           dp[i - 1][j - 1] + cost)  # Substitution

    return dp[len(str1)][len(str2)]

In [13]:
def orthographic_similarity(word1, word2):

    word1 = str(word1)
    word2 = str(word2)

    # Calculate similarity ratio between the two words
    d = levenshtein_distance(word1, word2)
    similarity = 1 - d/ max(len(word1), len(word2))

    return similarity

In [14]:
from difflib import SequenceMatcher

def sequence_matcher(word1, word2):

    word1 = str(word1)
    word2 = str(word2)

    # Calculate similarity ratio between the two words
    d = SequenceMatcher(None, word1, word2).ratio()

    return d

**Information Value**

In [15]:
def extract_words_and_embeddings(subwords, subword_embeddings):
    words = []
    word_embedding = []

    current_word = ""
    current_embedding = []

    for subword, embedding in zip(subwords, subword_embeddings):
        # Check if the subword starts with the special token 'Ġ'
        if subword.startswith('Ġ'):
            if current_word:
                words.append(current_word)
                word_embedding.append(sum(current_embedding) / len(current_embedding))

            # Reset current word and probability for the new word
            current_word = subword[1:]  # Remove 'Ġ' from the start
            current_embedding= [embedding]
        else:
            # Concatenate subwords to form the current word
            current_word += subword
            # Multiply probabilities for subwords within the same word
            current_embedding.append(embedding)

    # Add the last word and its probability
    if current_word:
        words.append(current_word)
        word_embedding.append(sum(current_embedding) / len(current_embedding))

    return words, word_embedding

In [16]:
def extract_words_and_probabilities(subwords, subword_probabilities):
    words = []
    word_probabilities = []

    current_word = ""
    current_probability = 1.0  # Initialize to 1.0 as we will multiply probabilities

    # Iterate through the subword probabilities
    for subword, probability in zip(subwords, subword_probabilities):
        # Check if the subword starts with the special token 'Ġ'
        if subword.startswith('Ġ'):
            # If we have a current word, add it to the list with its probability
            if current_word:
                words.append(current_word)
                word_probabilities.append(current_probability)

            # Reset current word and probability for the new word
            current_word = subword[1:]  # Remove 'Ġ' from the start
            current_probability = probability
        else:
            # Concatenate subwords to form the current word
            current_word += subword
            # Multiply probabilities for subwords within the same word
            current_probability *= probability

    # Add the last word and its probability
    if current_word:
        words.append(current_word)
        word_probabilities.append(current_probability)

    return words, word_probabilities

In [17]:
def calculate_word_information_values(sentence, vocabulary_df, similarity_function, model = model, tokenizer = tokenizer):

    #print(sentence)

    # calulate information value for one sentence
    words_list = []

    ce_iv = []
    nce_iv = []
    pt_iv = []
    od_iv = []

    words = sentence.split(' ')
    input_ids = tokenizer.encode(" ".join(words), return_tensors='pt')

    # Forward pass to get hidden states
    with torch.no_grad():
      outputs = model(input_ids, output_hidden_states=True)
      last_hidden_state = outputs.hidden_states[-1]  # This is the final layer's hidden state for each token

    decoded_subwords = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    decoded_words1, embeddings1 = extract_words_and_embeddings(decoded_subwords, last_hidden_state[0,:])

    # loop through all words in sentence
    for i in range(0, len(sentence.split(' '))):

      words = sentence.split(' ')
      word = words[i]
      #print(word)
      words_list.append(words[i])

      ce_iv.append(0)
      nce_iv.append(0)
      pt_iv.append(0)
      od_iv.append(0)

      embedding_word1 = embeddings1[i]

      vocab_df = vocabulary_df.sample(n=50, random_state=42).reset_index(drop=True)
      # loop through all vocabulary words
      for vord in vocab_df['word'].tolist():

        words[i] = vord
        # Tokenize the input sentence
        input_ids = tokenizer.encode(" ".join(words), return_tensors='pt')

        # Generate word probabilities using GPT-2 model
        with torch.no_grad():
            outputs = model(input_ids, output_hidden_states=True)
            logits = outputs.logits
            last_hidden_state = outputs.hidden_states[-1]  # This is the final layer's hidden state for each token

        # Extract probabilities for each word
        subword_probabilities = torch.softmax(logits, dim=-1).mean(dim=2)

        # Decode the tokens back to words
        decoded_subwords = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
        decoded_words2, probabilities = extract_words_and_probabilities(decoded_subwords, subword_probabilities[0])

        decoded_words2, embeddings2 = extract_words_and_embeddings(decoded_subwords, last_hidden_state[0,:])
        embedding_word2 = embeddings2[i]

        context_probability = probabilities[decoded_words2==vord].item()

        # distances
        contextual_embedding_distance = 1 - 0.5 * (F.cosine_similarity(embedding_word1, embedding_word2, dim=0).item() + 1)
        non_contextual_embedding_distance = 1 - non_context_embedding(word, vord, model, tokenizer)
        pos_tag_distance = 1 - pos_tags_similarity(word, vord, sentence, i)
        orthographic_distance = 1 - orthographic_similarity(word, vord)

        ce_iv[i] += contextual_embedding_distance * context_probability
        nce_iv[i] += non_contextual_embedding_distance * context_probability
        pt_iv[i] += pos_tag_distance * context_probability
        od_iv[i] += orthographic_distance * context_probability

    return words_list, ce_iv, nce_iv, pt_iv, od_iv

In [18]:
words_list = []
target_sentence_list = []
ce_iv_list = []
nce_iv_list = []
pt_iv_list = []
od_iv_list = []

# Save the DataFrame to a CSV file
csv_file_path = "/content/drive/MyDrive/PhD/information_value.csv"

for i in range(57,len(target_sentences_df)):
  sentence = target_sentences_df['Text'][i].lower()
  print(i)
  words, ce_ivs, nce_ivs, pt_ivs, od_ivs = calculate_word_information_values(sentence.strip(), vocabulary_df, pos_tags_similarity)

  for word, ce_iv, nce_iv, pt_iv, od_iv in zip(words, ce_ivs, nce_ivs, pt_ivs, od_ivs):
    words_list.append(word)
    target_sentence_list.append(i)
    ce_iv_list.append(ce_iv)
    nce_iv_list.append(nce_iv)
    pt_iv_list.append(pt_iv)
    od_iv_list.append(od_iv)

  # Create a DataFrame
  df = pd.DataFrame({
      'Sentence': target_sentence_list,
      'Word': words_list,
      'Context Embedding': ce_iv_list,
      'Non-context Embedding': nce_iv_list,
      'Pos-Tag': pt_iv_list,
      'Orthographic': od_iv_list
                     })
  df.to_csv(csv_file_path, index=False)

# Display the DataFrame
print(df)

57
58
59
60
61
62
63
64
65
66
    Sentence           Word  Context Embedding  Non-context Embedding  \
0         57             za       6.219793e-08           1.211129e-09   
1         57            dva       1.863104e-06           5.746504e-08   
2         57           dana       1.622229e-06           4.685987e-08   
3         57        počinje       9.902499e-07           5.078374e-08   
4         57          škola       1.040190e-06           5.301459e-08   
5         58             od       2.748790e-08           7.914371e-10   
6         58          sutra       2.163022e-06           6.225655e-08   
7         58            smo       3.416902e-06           7.828352e-08   
8         58           sami       1.900437e-06           5.467032e-08   
9         58              u       3.111979e-06           1.021707e-07   
10        58           kući       9.392866e-07           5.219573e-08   
11        59          petar       3.019864e-07           2.078094e-09   
12        59         